In [1]:
import gym
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import random
from IPython.display import clear_output
from collections import deque
from tqdm.notebook import tqdm
from collections import deque

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras.backend as K

tf.compat.v1.disable_eager_execution()

## Multi-Step Learning

### Main Ideas:

1. **Enhancing Temporal Credit Assignment**:
   - **Multi-Step Learning** extends traditional one-step Temporal Difference (TD) learning by considering multiple future steps when updating value estimates.
   - This approach provides a more accurate and robust estimation of the value function by aggregating rewards over several steps, improving the agent's ability to assign credit to actions that lead to long-term rewards.

2. **Balancing Bias and Variance**:
   - Single-step methods often suffer from high variance and biased estimates, especially in environments with delayed rewards.
   - **Multi-Step Learning** strikes a balance between bias and variance by incorporating information from multiple future steps, leading to more stable and reliable learning.

3. **Accelerating Learning**:
   - By utilizing rewards from multiple steps, multi-step methods can propagate information faster through the network, enhancing the speed of learning and convergence.
   - This acceleration is particularly beneficial in environments where rewards are sparse or delayed.

4. **Integration with Other Rainbow Components**:
   - **Multi-Step Learning** synergizes with other components of the Rainbow DQN framework, such as **Prioritized Experience Replay** and **Double DQN**, to further enhance learning efficiency and performance.
   - When combined, these techniques provide a more comprehensive approach to tackling complex reinforcement learning tasks.

### Structure of Multi-Step Learning:

- **n-Step Returns**:
  - The core concept of multi-step learning revolves around the computation of **n-step returns**, which aggregate rewards over \( n \) future steps.
  - The n-step return \( G_t^{(n)} \) at time \( t \) is defined as:
    $$
    G_t^{(n)} = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \dots + \gamma^{n-1} R_{t+n} + \gamma^n \max_{a'} Q(S_{t+n}, a'; \theta)
    $$
    where \( R_{t+i} \) is the reward received at step \( t+i \), \( \gamma \) is the discount factor, and \( \theta \) represents the network parameters.

- **Bootstrapping**:
  - After accumulating rewards over \( n \) steps, the estimate bootstraps from the value of the state at step \( t+n \), providing a balance between empirical returns and estimated values.

- **Experience Replay Integration**:
  - Multi-step returns are stored in the replay buffer alongside other experiences, allowing the agent to sample and learn from diverse temporal segments of its experience.

### Why Multi-Step Learning?

1. **Improved Temporal Credit Assignment**:
   - By considering multiple future rewards, multi-step methods can better associate actions with their long-term consequences, leading to more accurate policy updates.

2. **Enhanced Sample Efficiency**:
   - Utilizing information from multiple steps increases the amount of information extracted from each experience, making the learning process more sample-efficient.

3. **Faster Convergence**:
   - Multi-step updates can accelerate the propagation of reward signals through the network, leading to quicker convergence compared to single-step methods.

4. **Robustness to Reward Delays**:
   - In environments where rewards are delayed, multi-step learning helps in bridging the gap between actions and their eventual rewards, improving the agent's ability to learn optimal strategies.

### Advantages of Multi-Step Learning

1. **Better Credit Assignment**:
   - Facilitates more accurate attribution of rewards to the actions that led to them, enhancing the quality of policy updates.

2. **Increased Learning Speed**:
   - Aggregating rewards over multiple steps allows the agent to learn from a richer set of experiences, speeding up the overall learning process.

3. **Reduced Variance in Updates**:
   - By averaging rewards over multiple steps, multi-step methods can reduce the variance in value estimates, leading to more stable learning.

4. **Flexibility in Trade-Offs**:
   - The choice of \( n \) allows for tuning the balance between bias and variance, enabling adaptation to different types of environments and tasks.

### Implementation Details

- **Choosing the Step Size \( n \)**:
  - The step size \( n \) determines how many future steps are considered when computing the return. Common choices range from \( n=3 \) to \( n=5 \), but this can be tuned based on the specific environment and task requirements.

- **n-Step Transition Storage**:
  - When storing experiences in the replay buffer, multi-step transitions are stored as tuples containing the state, action, accumulated n-step return, and the state at step \( t+n \).

- **Bootstrapped Q-Value Updates**:
  - The network is updated using the n-step returns instead of the traditional one-step TD targets, enhancing the learning signal.

- **Integration with Prioritized Experience Replay**:
  - Multi-step returns can be combined with **Prioritized Experience Replay** by assigning priorities based on the n-step TD error, further emphasizing informative experiences.

### Conclusion

**Multi-Step Learning** is a pivotal component of the **Rainbow DQN** framework, enhancing the agent's ability to assign credit over extended temporal horizons and accelerating the learning process. By integrating multi-step returns with other advanced techniques like **Dueling Networks**, **Double DQN**, and **Prioritized Experience Replay**, Rainbow DQN achieves superior performance, stability, and sample efficiency in complex reinforcement learning environments. While multi-step learning introduces additional computational and implementation complexities, the benefits it offers in terms of improved credit assignment and faster convergence make it an indispensable tool in the arsenal of modern deep reinforcement learning strategies.


In [2]:
class N_step_Agent:
    def __init__(
            self,
            observation_space,
            action_space,
            gamma = 0.99,
            lr = 0.001,
            maxlen = 50000,
            n_step = 5,):
        
        self.observation_space = observation_space
        self.action_space = action_space
        self.gamma = gamma
        self.lr = lr
        self.buffer = deque(maxlen=maxlen)
        self.n_step = n_step
        self.n_step_memory = deque(maxlen=self.n_step)

        self.model = self.build_model(name = "model")
        self.target_model = self.build_model(name = "target")
        self.update_target()

    def build_model(self, name):
        model = keras.Sequential(name=name)
        model.add(layers.Input(shape=self.observation_space))
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dense(self.action_space, activation='linear'))

        optimizer = keras.optimizers.Adam(learning_rate=self.lr, clipvalue=1.0) 
        model.compile(
            optimizer=optimizer,
            loss='mse'
        )
        return model

    def remember(self, s, a, r, s_, t):
        self.n_step_memory.append((s,a,r,s_,t))

        if len(self.n_step_memory) < self.n_step:
            return
        
        R, flag = 0, False 
        for idx, (_, _, r, _, t) in enumerate(self.n_step_memory):
            R += (self.gamma ** idx) * r

            if t:
                flag = True
                break
        state_n, action_n, _, _, _ = self.n_step_memory[0]
        _, _, _, next_state_n, _ = self.n_step_memory[-1]
        t = flag

        self.buffer.append((state_n, action_n, R, next_state_n, t))

        if flag :
            self.n_step_memory.clear()

    def get_batch(self, batch_size):
        if len(self.buffer) < batch_size:
            return None
        
        batch = random.sample(self.buffer, k=batch_size)
        
        S = np.array([val[0] for val in batch])
        A = np.array([val[1] for val in batch])
        R = np.array([val[2] for val in batch])
        S_ = np.array([val[3] for val in batch])
        T = np.array([val[4] for val in batch])

        return S, A, R, S_, T

    def update_target(self):
        self.target_model.set_weights(self.model.get_weights())

    def predict(self, observation, target = False):
        if target:
            return self.target_model.predict(np.array([observation]), verbose = False)[0]
        return self.model.predict(np.array([observation]), verbose= False)[0]
    
    def predict_action(self, observation, epsilon=0.05):
        if np.random.random() < epsilon:
            return np.random.randint(self.action_space)
        return np.argmax(self.model.predict(np.array([observation]), verbose=False)[0])
    
    def train(self, batch_size):
        if len(self.buffer) < batch_size:
            return
        
        S, A, R, S_, T = self.get_batch(batch_size)
        
        Q_values = self.model.predict(S, verbose=False)
        
        Q_next = self.target_model.predict(S_, verbose=False)
        
        max_Q_next = np.max(Q_next, axis = 1)

        targets = Q_values.copy()
        targets[np.arange(batch_size), A] = R + (self.gamma ** self.n_step) * max_Q_next * (1 - T)

        self.model.fit(S, targets, verbose=False)

        

In [3]:
env = gym.make("LunarLander-v2")
agent = N_step_Agent(env.observation_space.shape, env.action_space.n)

episodes = 2000
max_t = 500
e=0.05
scores = []
avgs = []
time = 0
frequency = 400

for episode in range(episodes):
    observation, _ = env.reset()
    
    score = 0
    for t in range(max_t):
        time += 1
        s = observation
        a = agent.predict_action(s, e)
        observation, reward, terminated, _, _ = env.step(a)
        score +=  reward
        s_ = observation
        agent.remember(s, a, reward, s_, terminated)
        agent.train(batch_size=32)
        if time % frequency == 0:
            agent.update_target()
        if terminated:
            break
    scores.append(score)
    avgs.append(np.sum(scores[-50:])/len(scores[-50:]))
    print(f"episode: {episode}, e: {e}, t: {t}, score: {score : .2f}, avg score: {avgs[-1]: .2f}")
    if avgs[-1] >= 200:
        break

c:\Users\admin\anaconda3\envs\test\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
c:\Users\admin\anaconda3\envs\test\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


episode: 0, e: 0.05, t: 59, score: -77.66, avg score: -77.66
episode: 1, e: 0.05, t: 120, score: -373.37, avg score: -225.52
episode: 2, e: 0.05, t: 75, score: -274.83, avg score: -241.95
episode: 3, e: 0.05, t: 88, score: -218.49, avg score: -236.09
episode: 4, e: 0.05, t: 85, score: -288.56, avg score: -246.58
episode: 5, e: 0.05, t: 178, score: -198.34, avg score: -238.54
episode: 6, e: 0.05, t: 320, score: -336.32, avg score: -252.51
episode: 7, e: 0.05, t: 374, score: -160.18, avg score: -240.97
episode: 8, e: 0.05, t: 499, score: -83.09, avg score: -223.43
episode: 9, e: 0.05, t: 394, score: -177.38, avg score: -218.82
episode: 10, e: 0.05, t: 252, score: -374.30, avg score: -232.96
episode: 11, e: 0.05, t: 499, score: -171.91, avg score: -227.87
episode: 12, e: 0.05, t: 499, score: -84.55, avg score: -216.84
episode: 13, e: 0.05, t: 499, score: -36.50, avg score: -203.96
episode: 14, e: 0.05, t: 499, score: -89.16, avg score: -196.31
episode: 15, e: 0.05, t: 343, score: -167.59,